In [1]:
import pandas as pd
import scipy.sparse as sp
import numpy as np

In [2]:
dataset = pd.read_csv("../Input/interactions_and_impressions.csv")

dataset = dataset.drop(columns=['Impressions'])

datasetCOO = sp.coo_matrix((dataset["Data"].values, (dataset["UserID"].values, dataset["ItemID"].values)))
userIDS = dataset['UserID'].unique()
itemIDS = dataset['ItemID'].unique()

C:\Users\Mike\AppData\Local\Temp\ipykernel_46728\3063296559.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("../Input/interactions_and_impressions.csv")


In [3]:
datasetGrouped = dataset.groupby(['UserID', 'ItemID', 'Data']).size()
datasetGrouped

UserID  ItemID  Data
0       11      1        1
        21      0        3
                1        9
        22      1       15
        24      1        1
                        ..
41628   20448   0        1
        20896   1        1
        21506   1        1
        22882   0        1
        23757   0        1
Length: 1715198, dtype: int64

In [6]:
URM = np.zeros((len(userIDS), len(itemIDS)), dtype=int)
ones_list = np.zeros((len(userIDS), len(itemIDS)), dtype=int)
negative_users = np.zeros((len(userIDS), 2), dtype=int)

for x in range(len(datasetCOO.data)):
    if datasetCOO.data[x] == 0:
        if URM[datasetCOO.row[x]][datasetCOO.col[x]] <= 4:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(5)
            if negative_users[datasetCOO.row[x]][0] == 1 and negative_users[datasetCOO.row[x]][1] == datasetCOO.col[x]:
                negative_users[datasetCOO.row[x]][0] = 0
        else:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(7)
    elif datasetCOO.data[x] == 1:
        if URM[datasetCOO.row[x]][datasetCOO.col[x]] == 7:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(5)
        elif negative_users[datasetCOO.row[x]][0] == 0:
            negative_users[datasetCOO.row[x]][0] = 1
            negative_users[datasetCOO.row[x]][1] = datasetCOO.col[x]
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = -1
        elif URM[datasetCOO.row[x]][datasetCOO.col[x]] == 0:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = 2
            ones_list[datasetCOO.row[x]][datasetCOO.col[x]] += 1
        elif URM[datasetCOO.row[x]][datasetCOO.col[x]] >= 2 and ones_list[datasetCOO.row[x]][datasetCOO.col[x]] < 3:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] += 1
            ones_list[datasetCOO.row[x]][datasetCOO.col[x]] += 1
        elif URM[datasetCOO.row[x]][datasetCOO.col[x]] > 1 and URM[datasetCOO.row[x]][datasetCOO.col[x]] != 5 and URM[datasetCOO.row[x]][datasetCOO.col[x]] != 7 and ones_list[datasetCOO.row[x]][datasetCOO.col[x]] >= 3:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = URM[datasetCOO.row[x]][datasetCOO.col[x]] - 1

In [7]:
URM

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
URM = np.zeros((len(userIDS), len(itemIDS)), dtype=int)
ones_list = np.zeros((len(userIDS), len(itemIDS)), dtype=int)
for x in range(len(datasetCOO.data)):
    if datasetCOO.data[x] == 0:
        if URM[datasetCOO.row[x]][datasetCOO.col[x]] <= 4:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(5)
        else:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(7)
    elif datasetCOO.data[x] == 1:
        if URM[datasetCOO.row[x]][datasetCOO.col[x]] == 7:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = int(5)
        elif URM[datasetCOO.row[x]][datasetCOO.col[x]] == 0:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = 2
            ones_list[datasetCOO.row[x]][datasetCOO.col[x]] += 1
        elif URM[datasetCOO.row[x]][datasetCOO.col[x]] >= 2 and ones_list[datasetCOO.row[x]][datasetCOO.col[x]] < 3:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] += 1
            ones_list[datasetCOO.row[x]][datasetCOO.col[x]] += 1
        elif URM[datasetCOO.row[x]][datasetCOO.col[x]] > 1 and URM[datasetCOO.row[x]][datasetCOO.col[x]] != 5 and URM[datasetCOO.row[x]][datasetCOO.col[x]] != 7 and ones_list[datasetCOO.row[x]][datasetCOO.col[x]] >= 3:
            URM[datasetCOO.row[x]][datasetCOO.col[x]] = URM[datasetCOO.row[x]][datasetCOO.col[x]] - 1

URM

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])